In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
def preprocess(text):
# Preprocessing 
 
    new_text = [
    ]
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

load data

In [ ]:
!pip install transformers

     |████████████████████████████████| 2.5MB 6.7MB/s 
     |████████████████████████████████| 901kB 39.8MB/s 
     |████████████████████████████████| 3.3MB 51.0MB/s 


load models

In [ ]:
import torch

def find_cuda_device():
    if torch.cuda.is_available():
        device = torch.cuda.current_device()
        print('Current device:', torch.cuda.get_device_name(device))
    else:
        print('Failed to find GPU. Will use CPU.')
        device = 'cpu'
    return device

device = find_cuda_device()

Current device: Tesla T4


In [ ]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import softmax
import csv
import urllib.request




tasks = [ 'emotion', 'hate', 'irony', 'offensive', 'sentiment']
labels_list = []
models = []
tokenizers = []

#tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base")

for task in tasks:
    MODEL = f"cardiffnlp/twitter-roberta-base-{task}"
    #tok = f"cardiffnlp/twitter-roberta-base"
    tokenizer = AutoTokenizer.from_pretrained(MODEL)
    
    # download label mapping
    labels=[]
    mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
    with urllib.request.urlopen(mapping_link) as f:
        html = f.read().decode('utf-8').split("\n")
        csvreader = csv.reader(html, delimiter='\t')
    labels = [row[1] for row in csvreader if len(row) > 1]
    
    # get model
    model = AutoModelForSequenceClassification.from_pretrained(MODEL)
    model = model.to(device)
    model.save_pretrained(MODEL)

    # add to models
    models.append(model)
    labels_list.append(labels)
    tokenizers.append(tokenizer)

print(labels_list)


[['anger', 'joy', 'optimism', 'sadness'], ['not-hate', 'hate'], ['non_irony', 'irony'], ['not-offensive', 'offensive'], ['negative', 'neutral', 'positive']]


Processing

In [ ]:
# load data 
import pandas as pd
from datetime import datetime, timedelta

dateparse = lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S%z')

file = 'starbucks_labeled'

df = pd.read_csv(f'/content/drive/MyDrive/MRP Group 7/Data_eval_train/{file}.csv', parse_dates=['Datetime'], date_parser=dateparse, index_col=0)
df['Datetime'] =  df['Datetime'].dt.date

df['Text'] = df.Text.apply(lambda x: preprocess(x))

start_date = df['Datetime'].min()
end_date = df['Datetime'].max()
date = start_date

df_freq  = df.groupby(['Datetime', 'canceled'])['Datetime'].describe().reset_index()
{*df_freq}
df_freq

,Datetime,canceled,count,unique,top,freq
0,2020-06-06,False,7765,1,2020-06-06,7765
1,2020-06-07,False,7816,1,2020-06-07,7816
2,2020-06-08,False,7834,1,2020-06-08,7834
3,2020-06-09,False,8519,1,2020-06-09,8519
4,2020-06-10,True,19732,1,2020-06-10,19732
5,2020-06-11,True,34779,1,2020-06-11,34779
6,2020-06-12,True,50998,1,2020-06-12,50998
7,2020-06-13,True,16009,1,2020-06-13,16009
8,2020-06-14,True,8791,1,2020-06-14,8791
9,2020-06-15,True,9083,1,2020-06-15,9083


In [ ]:
from tqdm.notebook import tqdm
from pandas.core.common import flatten

number_of_tweets_per_day = 2500


# create 
df_aggregated = pd.DataFrame()
for x in labels_list:
  for y in labels_list:
    df_aggregated[y] = 0
df_aggregated['frequency'] = 0
df_aggregated['label'] = 0

date_range = pd.date_range(start_date,end_date)

print('Total progress:')

for date in tqdm(date_range):

  print(f'\nStart with day {date}')

  # get all tweets by date 
  data = df.loc[df['Datetime'] == date]['Text'].to_numpy()

  # select subset max 2500
  if number_of_tweets_per_day<len(data):
    number_of_rows = data.shape[0]
    random_indices = np.random.choice(number_of_rows, size=number_of_tweets_per_day, replace=False)
    data = data[random_indices]
  outputs_list = []

  #iterate through every model
  for i in tqdm(range(len(models))):
    print(f'Start with {tasks[i]} model')

    # encode inputs using model specific tokenizer
    encoded_inputs = [tokenizers[i](x, return_tensors='pt') for x in data]
    outputs = []

    # predict for every data point
    for encoded_input in tqdm(encoded_inputs):
      encoded_input = encoded_input.to(device)
      output = models[i](**encoded_input)
      logits = output[0][0].cpu().detach().numpy()
      scores = softmax(logits)
      outputs.append(scores)
    outputs = np.array(outputs)

    # average scores
    output_average = np.mean(outputs, axis=0)
    outputs_list.append(output_average)

  # add new row with average scores  
  outputs_list.append(df_freq.loc[df_freq['Datetime'] == date]['freq'])
  outputs_list.append(df_freq.loc[df_freq['Datetime'] == date]['canceled'])

  # if it crashes here 
  df_aggregated.loc[date] = list(flatten(outputs_list))

# 0-max normalization
df_aggregated['frequency_normalized']=(df_aggregated['frequency'])/(df_aggregated['frequency'].max()) 
df_aggregated.to_csv(f'/content/drive/MyDrive/MRP Group 7/Data_eval_train/{file}_results.csv')

#ranking = np.argsort(scores)
#ranking = ranking[::-1]
#for i in range(scores.shape[0]):
#    l = labels[ranking[i]]
#    s = scores[ranking[i]]
#    print(f"{i+1}) {l} {np.round(float(s), 4)}")

Total progress:



Start with day 2020-06-06 00:00:00


Start with emotion model



Start with hate model



Start with irony model


...

Start with emotion model



Start with hate model



Start with irony model



Start with offensive model



Start with sentiment model
